# Continuous Control - 20 agents

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [2]:
env = UnityEnvironment(file_name='Reacher20.app')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [5]:
# env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
# states = env_info.vector_observations                  # get the current state (for each agent)
# scores = np.zeros(num_agents)                          # initialize the score (for each agent)
# while True:
#     actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
#     actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
#     env_info = env.step(actions)[brain_name]           # send all actions to tne environment
#     next_states = env_info.vector_observations         # get next state (for each agent)
#     rewards = env_info.rewards                         # get reward (for each agent)
#     dones = env_info.local_done                        # see if episode finished
#     scores += env_info.rewards                         # update the score (for each agent)
#     states = next_states                               # roll over states to next time step
#     if np.any(dones):                                  # exit loop if episode finished
#         break
# print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

random_seed = 2. Average Score: 0.0982<br>
random_seed = 3. Average Score: 0.0793

## Make separate agents for every arm

In [ ]:
%%time 
from ddpg_agent20 import Agent, ReplayBuffer
from collections import deque
import torch


# note: the environment max_t maxes out at 1000
def ddpg(n_episodes=300, max_t=1000, print_every=100, load_checkpoints=True, checkpoint=0):
    scores_deque = deque(maxlen=print_every)
    scores = []
    best_scores = []

    print('creating agents')
    # instantiate a list of agents using single shared replay buffer
    replay_buffer = ReplayBuffer(action_size, int(1e6), 64, 42)
    agents = [Agent(state_size, action_size, i+42, replay_buffer) for i in range(num_agents)]
    
    # restart from checkpointed files
    if load_checkpoints:
        idx = checkpoint
        agents = []
        for i in range(num_agents):
            agents.append(Agent(state_size, action_size, i, replay_buffer))
            agents[i].actor_local.load_state_dict(torch.load('checkpoint_actor{:02d}.pth'.format(idx)))
            agents[i].actor_target.load_state_dict(torch.load('checkpoint_actor{:02d}.pth'.format(idx)))
            agents[i].critic_local.load_state_dict(torch.load('checkpoint_critic{:02d}.pth'.format(idx)))
            agents[i].critic_target.load_state_dict(torch.load('checkpoint_critic{:02d}.pth'.format(idx)))

    argmax = lambda lst: lst.index(max(lst))
    argmin = lambda lst: lst.index(min(lst))
    
    bestest_score = 0
    
    for i_episode in range(1, n_episodes+1):
        
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations
        
        for i in range(num_agents): agents[i].reset()
        score = 0
        scores = [0]*num_agents

        print('\nEpisode {}'.format(i_episode))
        for t in range(max_t):
            actions = [agents[i].act(states[i]) for i in range(num_agents)]
            env_info = env.step(actions)[brain_name]       # send the action to the environment
            next_states = env_info.vector_observations     # get the next state
            rewards = env_info.rewards                     # get the reward
            dones = env_info.local_done                    # see if episode has finished
            for i in range(num_agents):
                agents[i].step(states[i], actions[i], rewards[i], next_states[i], dones[i])
                scores[i] += rewards[i]
            imax = argmax(scores)
            imin = argmax(scores)
            best_score = scores[imax]
            states = next_states
            score += sum(rewards) / len(rewards)
            print('\riteration: {}/{}\tavg score: {:.2e}\tbest agent: {}\tscore: {:.2e} '.format(t+1, max_t, score, imax, best_score), end="")

            if np.any(dones): break
        
        scores_deque.append(score)
        scores.append(score)
        best_scores.append(best_score)
                
        with open('scores.log', 'a') as fp:
            fp.write("{:d}\t{:2e}\t{:2e}\n".format(i_episode, score, best_score))

        print('\tdeque: {:.5e}'.format(np.mean(scores_deque)), end="")
        
        # checkpoint the best agent
        torch.save(agents[imax].actor_local.state_dict(), 'checkpoint_actor{:02d}.pth'.format(0))
        torch.save(agents[imax].critic_local.state_dict(), 'checkpoint_critic{:02d}.pth'.format(0))

        if best_score > bestest_score:
            bestest_score = best_score
            print('\ncheckpointing agent {:d} with score {:.2e}'.format(imax, best_score))
            torch.save(agents[imax].actor_local.state_dict(), 'checkpoint_actor{:02d}.pth'.format(99))
            torch.save(agents[imax].critic_local.state_dict(), 'checkpoint_critic{:02d}.pth'.format(99))
            
        if i_episode % print_every == 0:
            print('\rEpisode {}\tAverage Score: {:.5e}'.format(i_episode, np.mean(scores_deque)))
            
    return scores, best_scores

scores, best_scores = ddpg()

creating agents

Episode 1
iteration: 1000/1000	avg score: 9.77e-01	best agent: 15	score: 2.09e+00 	deque: 9.77500e-01
checkpointing agent 15 with score 2.09e+00

Episode 2
iteration: 1000/1000	avg score: 8.37e-01	best agent: 16	score: 1.72e+00 	deque: 9.07500e-01
Episode 3
iteration: 1000/1000	avg score: 1.14e+00	best agent: 4	score: 2.50e+00 	deque: 9.83667e-01
checkpointing agent 4 with score 2.50e+00

Episode 4
iteration: 1000/1000	avg score: 1.12e+00	best agent: 17	score: 2.70e+00 	deque: 1.01762e+00
checkpointing agent 17 with score 2.70e+00

Episode 5
iteration: 1000/1000	avg score: 8.53e-01	best agent: 16	score: 1.83e+00 	deque: 9.84700e-01
Episode 6
iteration: 1000/1000	avg score: 9.46e-01	best agent: 6	score: 1.93e+00 	deque: 9.78250e-01
Episode 7
iteration: 1000/1000	avg score: 1.00e+00	best agent: 11	score: 2.73e+00 	deque: 9.82000e-01
checkpointing agent 11 with score 2.73e+00

Episode 8
iteration: 1000/1000	avg score: 7.99e-01	best agent: 17	score: 1.88e+00 	deque: 9.5918

iteration: 1000/1000	avg score: 8.90e-01	best agent: 11	score: 2.21e+00 	deque: 9.85942e-01
Episode 79
iteration: 1000/1000	avg score: 1.06e+00	best agent: 10	score: 2.05e+00 	deque: 9.86854e-01
Episode 80
iteration: 1000/1000	avg score: 1.02e+00	best agent: 7	score: 2.01e+00 	deque: 9.87312e-01
Episode 81
iteration: 1000/1000	avg score: 1.06e+00	best agent: 2	score: 2.13e+00 	deque: 9.88241e-01
Episode 82
iteration: 1000/1000	avg score: 1.00e+00	best agent: 9	score: 1.96e+00 	deque: 9.88409e-01
Episode 83
iteration: 1000/1000	avg score: 9.71e-01	best agent: 18	score: 1.72e+00 	deque: 9.88199e-01
Episode 84
iteration: 1000/1000	avg score: 1.08e+00	best agent: 5	score: 1.86e+00 	deque: 9.89268e-01
Episode 85
iteration: 1000/1000	avg score: 8.97e-01	best agent: 0	score: 1.61e+00 	deque: 9.88182e-01
Episode 86
iteration: 1000/1000	avg score: 1.16e+00	best agent: 15	score: 2.73e+00 	deque: 9.90227e-01
Episode 87
iteration: 1000/1000	avg score: 9.16e-01	best agent: 10	score: 1.94e+00 	deque

iteration: 1000/1000	avg score: 1.16e+00	best agent: 10	score: 2.13e+00 	deque: 1.00368e+00
Episode 158
iteration: 1000/1000	avg score: 8.14e-01	best agent: 0	score: 2.04e+00 	deque: 1.00217e+00
Episode 159
iteration: 1000/1000	avg score: 9.67e-01	best agent: 1	score: 1.97e+00 	deque: 1.00285e+00
Episode 160
iteration: 1000/1000	avg score: 9.91e-01	best agent: 2	score: 1.83e+00 	deque: 1.00223e+00
Episode 161
iteration: 1000/1000	avg score: 1.02e+00	best agent: 13	score: 2.28e+00 	deque: 1.00080e+00
Episode 162
iteration: 1000/1000	avg score: 9.62e-01	best agent: 14	score: 1.95e+00 	deque: 1.00089e+00
Episode 163
iteration: 1000/1000	avg score: 9.10e-01	best agent: 15	score: 1.83e+00 	deque: 9.99675e-01
Episode 164
iteration: 1000/1000	avg score: 8.35e-01	best agent: 12	score: 1.69e+00 	deque: 9.99690e-01
Episode 165
iteration: 1000/1000	avg score: 9.91e-01	best agent: 9	score: 1.94e+00 	deque: 1.00048e+00
Episode 166
iteration: 1000/1000	avg score: 1.26e+00	best agent: 16	score: 3.32e

iteration: 1000/1000	avg score: 1.03e+00	best agent: 18	score: 1.92e+00 	deque: 9.80245e-01
Episode 237
iteration: 1000/1000	avg score: 8.87e-01	best agent: 5	score: 1.77e+00 	deque: 9.78230e-01
Episode 238
iteration: 1000/1000	avg score: 8.22e-01	best agent: 12	score: 2.31e+00 	deque: 9.75275e-01
Episode 239
iteration: 1000/1000	avg score: 9.44e-01	best agent: 0	score: 2.18e+00 	deque: 9.74670e-01
Episode 240
iteration: 1000/1000	avg score: 1.07e+00	best agent: 10	score: 3.27e+00 	deque: 9.72725e-01
Episode 241
iteration: 1000/1000	avg score: 9.29e-01	best agent: 5	score: 1.77e+00 	deque: 9.72170e-01
Episode 242
iteration: 1000/1000	avg score: 9.71e-01	best agent: 1	score: 2.24e+00 	deque: 9.72855e-01
Episode 243
iteration: 1000/1000	avg score: 9.71e-01	best agent: 13	score: 2.26e+00 	deque: 9.72235e-01
Episode 244
iteration: 1000/1000	avg score: 1.07e+00	best agent: 16	score: 2.12e+00 	deque: 9.74510e-01
Episode 245
iteration: 1000/1000	avg score: 8.52e-01	best agent: 13	score: 2.50e

iteration: 1000/1000	avg score: 9.05e-01	best agent: 1	score: 2.27e+00 	deque: 1.00378e+00
Episode 316
iteration: 1000/1000	avg score: 9.56e-01	best agent: 3	score: 1.77e+00 	deque: 1.00381e+00
Episode 317
iteration: 1000/1000	avg score: 1.09e+00	best agent: 9	score: 2.49e+00 	deque: 1.00390e+00
Episode 318
iteration: 1000/1000	avg score: 1.05e+00	best agent: 16	score: 2.75e+00 	deque: 1.00225e+00
Episode 319
iteration: 1000/1000	avg score: 1.11e+00	best agent: 13	score: 2.36e+00 	deque: 1.00457e+00
Episode 320
iteration: 1000/1000	avg score: 9.55e-01	best agent: 16	score: 2.05e+00 	deque: 1.00384e+00
Episode 321
iteration: 1000/1000	avg score: 1.13e+00	best agent: 19	score: 3.32e+00 	deque: 1.00483e+00
Episode 322
iteration: 1000/1000	avg score: 9.03e-01	best agent: 3	score: 3.02e+00 	deque: 1.00446e+00
Episode 323
iteration: 1000/1000	avg score: 7.68e-01	best agent: 10	score: 1.66e+00 	deque: 1.00023e+00
Episode 324
iteration: 1000/1000	avg score: 1.10e+00	best agent: 7	score: 1.98e+

## <strike>Single agent for all arms </strike>

In [ ]:
# from ddpg_agent import Agent
# from collections import deque
# import torch


# def ddpg(n_episodes=200, max_t=1000, print_every=100):
#     scores_deque = deque(maxlen=print_every)
#     scores = []

#     # instantiate 
#     agent = Agent(state_size=20*state_size, action_size=20*action_size, random_seed=4)
#     print(agent.action_size)
#     print(agent.state_size)

#     for i_episode in range(1, n_episodes+1):
        
#         env_info = env.reset(train_mode=True)[brain_name]
#         states = env_info.vector_observations.flatten()
#         print(len(states))
#         agent.reset()
#         score = 0

#         for t in range(max_t):
#             actions = agent.act(states)
#             env_info = env.step(actions)[brain_name]       # send the action to the environment
#             next_states = env_info.vector_observations     # get the next state
#             rewards = env_info.rewards                     # get the reward
#             dones = env_info.local_done                    # see if episode has finished
#             agent.step(states, actions, rewards, next_states, dones)
#             states = next_states
#             scores += rewards
#             if np.any(dones): break
        
#         scores_deque.append(score)
#         scores.append(score)
        
#         print('\rEpisode {}\tAverage Score: {:.4f}'.format(i_episode, np.mean(scores_deque)), end="")
#         for i in range(num_agents): torch.save(agents[i].actor_local.state_dict(), 'checkpoint_actor{:02d}.pth'.format(i))
#         for i in range(num_agents): torch.save(agents[i].critic_local.state_dict(), 'checkpoint_critic{:02d}.pth'.format(i))
#         if i_episode % print_every == 0:
#             print('\rEpisode {}\tAverage Score: {:.4f}'.format(i_episode, np.mean(scores_deque)))
            
#     return scores

# scores = ddpg()

### Plot the results

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

### 5. Watch the trained agent

In [ ]:
from dqn_agent import Agent
import time
time.sleep(30)

agent = Agent(state_size=state_size, action_size=action_size, random_seed=0)

# load the weights from file
agent.qnetwork_local.load_state_dict(torch.load('checkpoint.pth'))

env_info = env.reset(train_mode=False)[brain_name] # reset the environment
score = 0
while True:
    state = env_info.vector_observations[0]
    action = agent.act(state)
    env_info = env.step(action)[brain_name]
    reward = env_info.rewards[0]                   # get the reward
    score += reward                                # update the score
    done = env_info.local_done[0]
    if done: break
        
print("Final Score: {}".format(score))

When finished, you can close the environment.

In [ ]:
env.close()